In [2]:
import torch

In [3]:
a = torch.randn(1000)

In [6]:
b = torch.argsort(a, descending=True)
#c = torch.argsort(b)

In [9]:
a[b]

tensor([ 3.8198e+00,  3.0566e+00,  2.8545e+00,  2.5881e+00,  2.5787e+00,
         2.5631e+00,  2.5088e+00,  2.4963e+00,  2.4103e+00,  2.3860e+00,
         2.3393e+00,  2.3253e+00,  2.2364e+00,  2.2192e+00,  2.1951e+00,
         2.1190e+00,  2.0999e+00,  2.0897e+00,  2.0714e+00,  2.0468e+00,
         2.0348e+00,  1.9845e+00,  1.9514e+00,  1.9166e+00,  1.9151e+00,
         1.9022e+00,  1.9000e+00,  1.8922e+00,  1.8868e+00,  1.8695e+00,
         1.8613e+00,  1.8564e+00,  1.8105e+00,  1.8064e+00,  1.7909e+00,
         1.7682e+00,  1.7160e+00,  1.7093e+00,  1.6823e+00,  1.6813e+00,
         1.6783e+00,  1.6603e+00,  1.6411e+00,  1.6348e+00,  1.6297e+00,
         1.6216e+00,  1.6116e+00,  1.6113e+00,  1.6052e+00,  1.5980e+00,
         1.5975e+00,  1.5952e+00,  1.5914e+00,  1.5903e+00,  1.5476e+00,
         1.5344e+00,  1.5272e+00,  1.5233e+00,  1.5208e+00,  1.5177e+00,
         1.5145e+00,  1.5108e+00,  1.5091e+00,  1.4893e+00,  1.4770e+00,
         1.4723e+00,  1.4678e+00,  1.4665e+00,  1.4

In [2]:
import torch
import os
import json
import argparse
from tqdm import tqdm
from fastchat.model.model_adapter import (
    load_model,
    get_conversation_template,
)
import pandas as pd
import h5py
import argparse
import numpy as np
#from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM
#from ftmodule.model import HFlikemodel
from transformers import StoppingCriteria,StoppingCriteriaList
from script.query_llm_model_inference import generate_with_start_kvcache



print("loading csv files...........")
sentense_ids = pd.read_csv("data/unarXive_quantum_physics/unarXive_quantum_physics.clear.sections.id.csv")
print("done~!")
need_question_ids = list(range(len(sentense_ids)))

# print("loading finished ids.........")
# with open("data/unarXive.clear/query.question.results.good_questions.ids.json", 'r') as f:good_question_ids = json.load(f)
# print("done~!")
# good_question_ids=set(good_question_ids)
# need_question_ids=set(list(range(len(sentense_ids)))) - good_question_ids
# need_question_ids = list(need_question_ids)
# print(f"remain {len(need_question_ids)}/{len(sentense_ids)} items")

SAVEPATH = "data/unarXive_quantum_physics/question_results"
sectionsf = h5py.File('data/unarXive_quantum_physics/unarXive_quantum_physics.clear.sections.h5', 'r')
# abstractf = h5py.File('data/unarXive.clear/unarXive.clear.abstract.h5', 'r')
titlef    = h5py.File('data/unarXive_quantum_physics/unarXive_quantum_physics.clear.title.h5', 'r')

print("loading model...........")
#model_path = 'pretrain_weights/fast_t5'
model_path = 'pretrain_weights/vicuna-7b-v1.1'
model, tokenizer = load_model(
    model_path,
    "cuda",
    1,
    load_8bit=False
)


loading csv files...........


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


done~!
loading model...........


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
__id= 0
tokenizer.padding_side='left'
_id        = need_question_ids[__id]
data       = sentense_ids.iloc[_id]
paper_id   = data['paper_id']
sentence_id= data['section_num']


#abstract = abstractf.get(f'abstract/{paper_id}')[()].decode('utf-8')
title    = titlef.get(f'abstract/{paper_id}')[()].decode('utf-8')
sentense = sectionsf.get(f'{paper_id}/{sentence_id}')[()].decode('utf-8')

#sentense = " ".join(sentense.split(" ")[:max_length])
conv = get_conversation_template(model_path)
#qs = f"""Read below sentence and tell me its type. The answer should be one word and is one of type from ['Author List', 'Reference', 'Content', 'Meaningless']. There is the sentence \"{sentense}\" """
qs = f"""I have a specific paragraph from a scholarly paper named <{title}>. I need your help to formulate an insightful question based on the information given. The specific paragraph from the paper is:\n\"\"\"\n{sentense} \n\"\"\"\nFollowing the question, provide a one-sentence response that succinctly answers it. The response should be start with "What is". Make the response as short as possible. """
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
prompts = [conv.get_prompt()]
#print(prompts[0])
input_ids = tokenizer(prompts).input_ids
input_ids = torch.as_tensor(input_ids).cuda()
#prompts    = [get_prompt(_id)] #(B=1)
#input_ids_f= tokenizer(prompts).input_ids
# tokenizer.pad_token = tokenizer.unk_token
# inputs_ids = tokenizer.pad({'input_ids': input_ids_f},padding='longest',
#                             max_length=512,pad_to_multiple_of=8,return_attention_mask=False,
#                             return_tensors='pt').input_ids.cuda()
length = len(input_ids[0])
# with torch.no_grad():

In [19]:

#conv.append_message(conv.roles[1], None)

"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER:"

In [169]:
from fastchat.serve.inference import *

@torch.inference_mode()
def generate_with_start_kvcache(
    model,
    tokenizer,
    params: Dict,
    device: str,
    context_len: int,
    stream_interval: int = 2,
    judge_sent_end: bool = False,
    start_kvcache= None,
    return_kvcache:bool=False
):
    if hasattr(model, "device"):device = model.device

    # Read parameters
    prompt = params["prompt"]
    len_prompt = len(prompt)
    temperature = float(params.get("temperature", 1.0))
    repetition_penalty = float(params.get("repetition_penalty", 1.0))
    top_p = float(params.get("top_p", 1.0))
    top_k = int(params.get("top_k", -1))  # -1 means disable
    max_new_tokens = int(params.get("max_new_tokens", 256))
    echo = bool(params.get("echo", False))
    stop_str = params.get("stop", None)
    stop_token_ids = params.get("stop_token_ids", None) or []
    stop_token_ids.append(tokenizer.eos_token_id)

    logits_processor = prepare_logits_processor(
        temperature, repetition_penalty, top_p, top_k
    )
    input_ids = tokenizer(prompt).input_ids

    if model.config.is_encoder_decoder:
        max_src_len = context_len
    else:  # truncate
        max_src_len = context_len - max_new_tokens - 1

    input_ids = input_ids[-max_src_len:]

    output_ids = list(input_ids)
    input_echo_len = len(input_ids)

    if model.config.is_encoder_decoder:
        encoder_output = model.encoder(input_ids=torch.as_tensor([input_ids], device=device))[0]
        start_ids = torch.as_tensor(
            [[model.generation_config.decoder_start_token_id]],
            dtype=torch.int64,
            device=device,
        )
    out = None
    
    past_key_values = start_kvcache
    sent_interrupt = False
    for i in range(max_new_tokens):
        if i == 0:  # prefill
            if model.config.is_encoder_decoder:
                out = model.decoder(input_ids=start_ids,encoder_hidden_states=encoder_output,use_cache=True)
                logits = model.lm_head(out[0])
            else:
                
                cover_token = past_key_values[-1][0].shape[-2] if past_key_values is not None else 0
                out = model(torch.as_tensor([input_ids[cover_token:]], device=device), use_cache=True,past_key_values=past_key_values)
                logits = out.logits
            past_key_values = out.past_key_values
        else:  # decoding
            if model.config.is_encoder_decoder:
                out = model.decoder(
                    input_ids=torch.as_tensor([[token] if not sent_interrupt else output_ids],device=device,),
                    encoder_hidden_states=encoder_output,
                    use_cache=True,
                    past_key_values=past_key_values if not sent_interrupt else None,
                )
                sent_interrupt = False
                logits = model.lm_head(out[0])
            else:
                out = model(
                    input_ids=torch.as_tensor([[token] if not sent_interrupt else output_ids],device=device,),
                    use_cache=True,
                    past_key_values=past_key_values if not sent_interrupt else None,
                )
                sent_interrupt = False
                logits = out.logits
            past_key_values = out.past_key_values

        if logits_processor:
            if repetition_penalty > 1.0:
                tmp_output_ids = torch.as_tensor([output_ids], device=logits.device)
            else:
                tmp_output_ids = None
            last_token_logits = logits_processor(tmp_output_ids, logits[:, -1, :])[0]
        else:
            last_token_logits = logits[0, -1, :]

        if device == "mps":
            # Switch to CPU by avoiding some bugs in mps backend.
            last_token_logits = last_token_logits.float().to("cpu")

        if temperature < 1e-5 or top_p < 1e-8:  # greedy
            _, indices = torch.topk(last_token_logits, 2)
            tokens = [int(index) for index in indices.tolist()]
        else:
            probs = torch.softmax(last_token_logits, dim=-1)
            indices = torch.multinomial(probs, num_samples=2)
            tokens = [int(token) for token in indices.tolist()]
        token = tokens[0]
        output_ids.append(token)

        if token in stop_token_ids:
            stopped = True
        else:
            stopped = False
        
        # Yield the output tokens
        if stopped:
            break
    #print(len(output_ids))
    tmp_output_ids = output_ids[input_echo_len:]
    output = tokenizer.decode(
                tmp_output_ids,
                skip_special_tokens=True,
                spaces_between_special_tokens=False,
                clean_up_tokenization_spaces=True,
            )
    # Finish stream event, which contains finish reason
    if i == max_new_tokens - 1:
        finish_reason = "length"
    elif stopped:
        finish_reason = "stop"
    else:
        finish_reason = None
    
    output = {
        "text": output,
        "usage": {
            "prompt_tokens": input_echo_len,
            "completion_tokens": i,
            "total_tokens": input_echo_len + i,
        },
        "finish_reason": finish_reason,
    }

    if return_kvcache:
        output['last_kvcache'] = past_key_values
    return output
    

In [114]:
prefix_kvcache[-1][0].shape[-2]

43

In [154]:
conv = get_conversation_template(model_path)
conv.append_message(conv.roles[0], "")
print(conv.get_prompt() )
prefix_kvcache = generate_with_start_kvcache(model,tokenizer,{'prompt':conv.get_prompt() ,'max_length':128},'cuda',context_len=16000,return_kvcache=True)['last_kvcache']

A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER:


In [145]:
len(tokenizer.encode(prefix_prompt))

34

In [146]:
qs = "Who are you?"

In [161]:
qs = f"""I have a specific paragraph from a scholarly paper named <{title}>. I need your help to formulate an insightful question based on the information given. The specific paragraph from the paper is:\n\"\"\"\n{sentense} \n\"\"\"\nFollowing the question, provide a one-sentence response that succinctly answers it. The response should be start with "What is". Make the response as short as possible. """

In [170]:

conv = get_conversation_template(model_path)
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)
result = generate_with_start_kvcache(model,tokenizer,{'prompt':conv.get_prompt(),'max_new_tokens':32,'temperature':0,'top_k':1},'cuda',context_len=16000,return_kvcache=True)

In [176]:
result['last_kvcache'][0][0].shape

torch.Size([1, 32, 232, 128])

In [183]:
%%timeit
conv = get_conversation_template(model_path)
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], result["text"])
conv.append_message(conv.roles[0], "Provide another question")
conv.append_message(conv.roles[1], None)
generate_with_start_kvcache(model,tokenizer,{'prompt':conv.get_prompt(),'max_new_tokens':32,'temperature':0,'top_k':1},'cuda',context_len=16000)

490 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [184]:
%%timeit
conv = get_conversation_template(model_path)
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], result["text"])
conv.append_message(conv.roles[0], "Provide another question")
conv.append_message(conv.roles[1], None)
generate_with_start_kvcache(model,tokenizer,{'prompt':conv.get_prompt(),'max_length':32,'temperature':0,'top_k':1},'cuda',context_len=16000,start_kvcache=result['last_kvcache'])['text']

458 ms ± 8.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
result

{'text': 'What is the significance of the focus on "standards fields" and lack of reference to strings in the quantum theory of particles and fields?',
 'usage': {'prompt_tokens': 544, 'completion_tokens': 28, 'total_tokens': 572},
 'finish_reason': 'stop'}